SalesLT_ProductCategory

In [0]:
# Enable auto merge

spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")

In [0]:
%run "/Workspace/Utils/Utils"

In [0]:
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, StringType, TimestampType, StructType, StructField
)
from pyspark.sql.functions import count
from pyspark.sql.functions import col, count
from pyspark.sql.functions import count, desc

In [0]:
#Loading Table


df = spark.table("adlslmcompany_bronze.managed_bronze.saleslt_productcategory")

In [0]:
#Displaying table

df.limit(10).display()

In [0]:
#Checking for duplicated values

checkduplicates(df, "ProductCategoryID" )

In [0]:
def silver_clean_productcategory(df): 

    # Adds transformation Date column
    df = df.withColumn("silves_transformed_timestamp", F.current_timestamp())


        #Cast to ensure datatype
    df = df.select(
         F.col('ProductCategoryID').cast(IntegerType()).alias('ProductCategoryID'),
         F.col('ParentProductCategoryID').cast(IntegerType()).alias('ParentProductCategoryID'),
         F.col('Name').cast(StringType()).alias('Name'),
         F.col('rowguid').cast(StringType()).alias('rowguid'),
         F.col('ModifiedDate').cast(TimestampType()).alias('ModifiedDate'),
         F.col('bronze_ingestion_timestamp').cast(TimestampType()).alias('bronze_ingestion_timestamp'),
         F.col('silves_transformed_timestamp').cast(TimestampType()).alias('silves_transformed_timestamp'),
                 )
    

    return df

In [0]:
#Defining expected schema
expected_schema = StructType([
    StructField("ProductCategoryID", IntegerType(), False), 
    StructField("ParentProductCategoryID", IntegerType(), False),
    StructField("Name", StringType(), False),
    StructField("rowguid", StringType(), False),
    StructField("ModifiedDate", TimestampType(), False),
    StructField("bronze_ingestion_timestamp", TimestampType(), False),
    StructField("silves_transformed_timestamp", TimestampType(), False)      
                            ])

In [0]:
#Transforming

silver_df = silver_clean_productcategory(df)

In [0]:
#Comparing lenghts

compare_lengths(df, silver_df)

In [0]:
#Checking the schema 
_validate_schema(silver_df, expected_schema)

**IMPORTANT: Please note that this is a simulated project; the upsert operation will be executed within this notebook. In a production environment, a dedicated notebook containing only the function and validations would be developed. All function notebooks would be orchestrated by Azure Data Factory (ADF) pipelines or Azure Databricks (ADB) workflows. The method of upsert may vary based on the utilization of auto loader, streaming, or Change Data Feed (CDF).**

In [0]:
#Loading into the Silver Layer  

target_table= "saleslt_productcategory"   

schema = "managed_silver"

catalog = "adlslmcompany_silver"

primary_keys = ["ProductCategoryID"]


_upsert_silver_table(silver_df, target_table, primary_keys, schema, catalog )